In [1]:
import pandas as pd
import os
import urllib
import gzip
import reverse_geocoder as rg

In [12]:
%%cmd
mkdir data_ghcn
cd data_ghcn
curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt > ghcnd-stations.txt
curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz > 2020.csv.gz

Microsoft Windows [Version 10.0.22621.819]
(c) Microsoft Corporation. All rights reserved.

C:\Data\berkeley_classes\fa22\data200-grad-project>mkdir data_ghcn


A subdirectory or file data_ghcn already exists.



C:\Data\berkeley_classes\fa22\data200-grad-project>cd data_ghcn

C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt > ghcnd-stations.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  5784k      0  0:00:01  0:00:01 --:--:-- 5782k



C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>curl https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz > 2020.csv.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  154M  100  154M    0     0  25.2M      0  0:00:06  0:00:06 --:--:-- 32.1M



C:\Data\berkeley_classes\fa22\data200-grad-project\data_ghcn>

In [13]:
import gzip
import shutil
with gzip.open('./data_ghcn/2020.csv.gz', 'rb') as f_in:
    with open('./data_ghcn/2020.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [20]:
def get_stations(filename='data_ghcn/ghcnd-stations.txt'):
    df = pd.read_csv(filename, '/t', header=None)
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    return df

stations = get_stations()
display(stations)

C:\Users\Abatics\AppData\Local\Temp\ipykernel_10772\1085517504.py:7: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  stations = get_stations()
C:\Users\Abatics\AppData\Local\Temp\ipykernel_10772\1085517504.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, '/t', header=None)


,Station,Latitude,Longitude,Elevation
0,ACW00011604,17.1167,-61.7833,10.1
1,ACW00011647,17.1333,-61.7833,19.2
2,AE000041196,25.3330,55.5170,34.0
3,AEM00041194,25.2550,55.3640,10.4
4,AEM00041217,24.4330,54.6510,26.8
...,...,...,...,...
123178,ZI000067969,-21.0500,29.3670,861.0
123179,ZI000067975,-20.0670,30.8670,1095.0
123180,ZI000067977,-21.0170,31.5830,430.0
123181,ZI000067983,-20.2000,32.6160,1132.0


In [27]:
df = pd.read_csv("./data_ghcn/2020.csv", header=None).drop(columns=[4, 5, 6, 7])
df

,0,1,2,3
0,AE000041196,20200101,TMIN,168
1,AE000041196,20200101,PRCP,0
2,AE000041196,20200101,TAVG,211
3,AEM00041194,20200101,PRCP,0
4,AEM00041194,20200101,TAVG,217
...,...,...,...,...
36283671,WFM00091754,20201231,TMAX,310
36283672,WFM00091754,20201231,TMIN,260
36283673,WFM00091754,20201231,TAVG,282
36283674,WIM00060096,20201231,TMIN,156


In [28]:
df.columns = ['Station Name', 'Date', 'Element', 'Value']
df

,Station Name,Date,Element,Value
0,AE000041196,20200101,TMIN,168
1,AE000041196,20200101,PRCP,0
2,AE000041196,20200101,TAVG,211
3,AEM00041194,20200101,PRCP,0
4,AEM00041194,20200101,TAVG,217
...,...,...,...,...
36283671,WFM00091754,20201231,TMAX,310
36283672,WFM00091754,20201231,TMIN,260
36283673,WFM00091754,20201231,TAVG,282
36283674,WIM00060096,20201231,TMIN,156


In [36]:
prcp_tavg = df[(df['Element']=='TAVG') | (df['Element']=='PRCP')]

In [37]:
prcp_tavg_valid = prcp_tavg[prcp_tavg['Station Name']\
                            .isin(stations['Station'])]

In [44]:
prcp_tavg_valid = prcp_tavg_valid.pivot_table(index=['Date', 'Station Name'],
                            values='Value',
                            columns='Element').dropna()

In [45]:
prcp_tavg_valid.tail(10)

Element                PRCP   TAVG
Date     Station Name             
20201231 VMM00048845   28.0  149.0
         VMM00048848    0.0  161.0
         VMM00048855   10.0  205.0
         VMM00048887    3.0  257.0
         VMM00048914    0.0  262.0
         VMW00041010    3.0  232.0
         VMW00041011    0.0  232.0
         WA007878380   10.0  250.0
         WA010517310    8.0  252.0
         WF000917530    5.0  283.0